In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

In [1]:
# If you're not using Colab, you might need to upgrade jupyter notebook to avoid the following error:
# 'ImportError: IProgress not found. Please update jupyter and ipywidgets.'

! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension

# Please restart the kernel after running this cell

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [18]:
import os
import wget
from nemo.collections import nlp as nemo_nlp
from omegaconf import OmegaConf

# Language models

Natural Language Processing (NLP) field experienced a huge leap in recent years due to the concept of transfer learning enabled through pretrained language models.

[BERT](https://arxiv.org/abs/1810.04805), [RoBERTa](https://arxiv.org/abs/1907.11692), [Megatron-LM](https://arxiv.org/abs/1909.08053), and many other proposed language models achieve state-of-the-art results on many NLP tasks, such as:
* question answering
* sentiment analysis
* named entity recognition and many others.

In NeMo, most of the NLP models represent a pretrained language model followed by a Token Classification layer or a Sequence Classification layer or a combination of both. By changing the language model, you can improve the performance of your final model on the specific downstream task you are solving.

With NeMo you can use either pretrain a BERT model from your data or use a pretrained language model from [HuggingFace transformers](https://github.com/huggingface/transformers) or [Megatron-LM](https://github.com/NVIDIA/Megatron-LM) libraries.

Note: Megatron BERT is not supported in NeMo 1.5.0. Please use [NeMo 1.4.0](https://github.com/NVIDIA/NeMo/tree/r1.4.0) for Megatron BERT support.

Let's take a look at the list of available pretrained language models, note the complete list of HuggingFace model could be found at [https://huggingface.co/models](https://huggingface.co/models):


In [19]:
nemo_nlp.modules.get_pretrained_lm_models_list()

['bert-base-uncased',
 'bert-large-uncased',
 'bert-base-cased',
 'bert-large-cased',
 'bert-base-multilingual-uncased',
 'bert-base-multilingual-cased',
 'bert-base-chinese',
 'bert-base-german-cased',
 'bert-large-uncased-whole-word-masking',
 'bert-large-cased-whole-word-masking',
 'bert-large-uncased-whole-word-masking-finetuned-squad',
 'bert-large-cased-whole-word-masking-finetuned-squad',
 'bert-base-cased-finetuned-mrpc',
 'bert-base-german-dbmdz-cased',
 'bert-base-german-dbmdz-uncased',
 'cl-tohoku/bert-base-japanese',
 'cl-tohoku/bert-base-japanese-whole-word-masking',
 'cl-tohoku/bert-base-japanese-char',
 'cl-tohoku/bert-base-japanese-char-whole-word-masking',
 'TurkuNLP/bert-base-finnish-cased-v1',
 'TurkuNLP/bert-base-finnish-uncased-v1',
 'wietsedv/bert-base-dutch-cased',
 'distilbert-base-uncased',
 'distilbert-base-uncased-distilled-squad',
 'distilbert-base-cased',
 'distilbert-base-cased-distilled-squad',
 'distilbert-base-german-cased',
 'distilbert-base-multilingu

NLP models for downstream tasks use `get_lm_model` helper function to easily switch between language models from the list above to another:

In [20]:
# use any pretrained model name from the list above
nemo_nlp.modules.get_lm_model(pretrained_model_name='distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertEncoder: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
-

DistilBertEncoder(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear

All NeMo [NLP models](https://github.com/NVIDIA/NeMo/tree/main/examples/nlp) have an associated config file. As an example, let's examine the config file for the Named Entity Recognition (NER) model (more details about the model and the NER task could be found [here](https://github.com/NVIDIA/NeMo/blob/stable/tutorials/nlp/Token_Classification_Named_Entity_Recognition.ipynb)).

In [21]:
MODEL_CONFIG = "token_classification_config.yaml"
BRANCH = 'main'
# download the model's configuration file 
if not os.path.exists(MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/token_classification/conf/' + MODEL_CONFIG)
else:
    print ('Config file already exists')

Config file already exists


In [22]:
# this line will print the entire config of the model
config = OmegaConf.load(MODEL_CONFIG)
print(OmegaConf.to_yaml(config))

pretrained_model: null
trainer:
  gpus: 1
  num_nodes: 1
  max_epochs: 5
  max_steps: null
  accumulate_grad_batches: 1
  gradient_clip_val: 0.0
  precision: 16
  accelerator: ddp
  checkpoint_callback: false
  logger: false
  log_every_n_steps: 1
  val_check_interval: 1.0
  resume_from_checkpoint: null
exp_manager:
  exp_dir: null
  name: token_classification_model
  create_tensorboard_logger: true
  create_checkpoint_callback: true
model:
  label_ids: null
  class_labels:
    class_labels_file: label_ids.csv
  dataset:
    data_dir: ???
    class_balancing: null
    max_seq_length: 128
    pad_label: O
    ignore_extra_tokens: false
    ignore_start_end: false
    use_cache: true
    num_workers: 2
    pin_memory: false
    drop_last: false
  train_ds:
    text_file: text_train.txt
    labels_file: labels_train.txt
    shuffle: true
    num_samples: -1
    batch_size: 64
  validation_ds:
    text_file: text_dev.txt
    labels_file: labels_dev.txt
    shuffle: false
    num_samples: -

For this tutorial, we are interested in the language_model part of the Named Entity Recognition Model.

In [23]:
print(OmegaConf.to_yaml(config.model.language_model))

pretrained_model_name: bert-base-uncased
lm_checkpoint: null
config_file: null
config: null
nemo_file: null



There might be slight differences from one model to another, but most of them have the following important parameters associated with the language model:
* `pretrained_model_name` - a name of the pretrained model from either HuggingFace or Megatron-LM libraries, for example, bert-base-uncased or megatron-bert-345m-uncased.
* `lm_checkpoint` - a path to the pretrained model checkpoint if, for example, you trained a BERT model with your data
* `config_file` -  path to the model configuration file
* `config` or `config_dict` - path to the model configuration dictionary

To modify the default language model, specify the desired language model name with the `model.language_model.pretrained_model_name` argument, like this:

In [24]:
config.model.language_model.pretrained_model_name = 'roberta-base'

and then start the training as usual (please see [tutorials/nlp](https://github.com/NVIDIA/NeMo/tree/main/tutorials/nlp) for more details about training of a particular model). 

You can also provide a pretrained language model checkpoint and a configuration file if available.

Note, that `pretrained_model_name` is used to set up both Language Model and Tokenizer.

All the above holds for both HuggingFace and Megatron-LM pretrained language models. Let's separately examine some specifics of finetuning with Megatron-LM and HuggingFace models.

# Downstream tasks with Megatron and BioMegatron Language Models

[Megatron-LM: Training Multi-Billion Parameter Language Models Using Model Parallelism](https://arxiv.org/abs/1909.08053) is a large, powerful transformer developed by the Applied Deep Learning Research team at NVIDIA. More details could be found at [Megatron-LM github repo](https://github.com/NVIDIA/Megatron-LM).

Note: Megatron BERT is not supported in NeMo 1.5.0. Please use [NeMo 1.4.0](https://github.com/NVIDIA/NeMo/tree/r1.4.0) for Megatron BERT support.

To see the list of available Megatron-LM models in NeMo, run:

In [25]:
nemo_nlp.modules.get_megatron_lm_models_list()

AttributeError: module 'nemo.collections.nlp.modules' has no attribute 'get_megatron_lm_models_list'

If you want to use one of the available Megatron-LM models, specify its name with `model.language_model.pretrained_model_name` argument, for example:

In [ ]:
#config.model.language_model.pretrained_model_name = 'megatron-bert-345m-uncased'

If you have a different checkpoint or a model configuration file, use these general Megatron-LM model names:
* `megatron-bert-uncased` or 
* `megatron-bert-cased` 

and provide associated bert_config and bert_checkpoint files, as follows:

`model.language_model.pretrained_model_name=megatron-bert-uncased \
model.language_model.lm_checkpoint=<PATH_TO_CHECKPOINT> \
model.language_model.config_file=<PAHT_TO_CONFIG>`
 
 or 
 
`model.language_model.pretrained_model_name=megatron-bert-cased \
model.language_model.lm_checkpoint=<PATH_TO_CHECKPOINT> \
model.language_model.config_file=<PAHT_TO_CONFIG>`

The general Megatron-LM model names are used to download the correct vocabulary file needed to setup the model correctly. Note, the data preprocessing and model training is done in NeMo. Megatron-LM has its own set of training arguments (including tokenizer) that are ignored during finetuning in NeMo. Please see downstream task [config files and training scripts](https://github.com/NVIDIA/NeMo/tree/main/examples/nlp) for all NeMo supported arguments.

## Download pretrained model

With NeMo, the original and domain-specific Megatron-LM BERT models and model configuration files will be downloaded automatically, but they also could be downloaded with the links below:

[Megatron-LM BERT Uncased 345M (~345M parameters): https://ngc.nvidia.com/catalog/models/nvidia:megatron_bert_345m](https://ngc.nvidia.com/catalog/models/nvidia:megatron_bert_345m/files?version=v0.1_uncased)

[Megatron-LM BERT Cased 345M (~345M parameters): https://ngc.nvidia.com/catalog/models/nvidia:megatron_bert_345m](https://ngc.nvidia.com/catalog/models/nvidia:megatron_bert_345m/files?version=v0.1_cased)

[BioMegatron-LM BERT Cased 345M (~345M parameters): https://ngc.nvidia.com/catalog/models/nvidia:biomegatron345mcased](https://ngc.nvidia.com/catalog/models/nvidia:biomegatron345mcased)

[BioMegatron-LM BERT Uncased 345M (~345M parameters)](https://ngc.nvidia.com/catalog/models/nvidia:biomegatron345muncased): https://ngc.nvidia.com/catalog/models/nvidia:biomegatron345muncased

# Using any HuggingFace Pretrained Model

Currently, there are 4 HuggingFace language models that have the most extensive support in [NeMo](https://github.com/NVIDIA/NeMo/tree/main/nemo/collections/nlp/modules/common/huggingface): 

* BERT
* RoBERTa
* ALBERT
* DistilBERT

As was mentioned before, just set `model.language_model.pretrained_model_name` to the desired model name in your config and get_lm_model() will take care of the rest.

If you want to use another language model from [https://huggingface.co/models](https://huggingface.co/models), use HuggingFace API directly in NeMo.
More details on model training could be found at [tutorials](https://github.com/NVIDIA/NeMo/tree/main/tutorials).